# Code for AISTATS submission (Reconstructing Test Labels from Noisy Loss Functions)
## multi-query label inference with log-loss and softmax-loss on CIFAR

### Experiment setup

In [1]:
import os
import sys
import json
import pandas as pd
import numpy as np
from collections import Counter
import math
import time
import warnings 
import random
warnings.filterwarnings('ignore')

np.random.seed(12345)

# CIFAR-10
(http://www.cs.toronto.edu/~kriz/cifar.html) CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images

In [2]:
cifar_label = [0,1,2,3,4,5,6,7,8,9] * 6000
random.shuffle(cifar_label)

In [3]:
print(cifar_label[:10])
Counter(cifar_label)

[7, 3, 2, 0, 7, 0, 8, 4, 4, 9]


Counter({7: 6000,
         3: 6000,
         2: 6000,
         0: 6000,
         8: 6000,
         4: 6000,
         9: 6000,
         6: 6000,
         5: 6000,
         1: 6000})

## utility functions in the experiments

In [4]:
# function to compute the k-ary-logloss score
def k_ary_logloss(labeling=[], prediction=np.array([[]])):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    Returns the k-ary-logloss score
    """
    N = len(labeling)
    loss = 0
    for i in range(N):
        cur_pred = prediction[i, labeling[i]]
        loss -= math.log(cur_pred)
    return loss / N

In [5]:
# function to compute the sigmoid loss
def sigmoid_loss(labeling=[], prediction=np.array([[]])):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    Returns the sigmoid loss score
    """
    N = len(labeling)
    loss = 0
    for i in range(N):
        cur_pred = prediction[i, labeling[i]]
        sig = 1 / (1 + np.exp(-cur_pred))
        loss -= math.log(sig)
    return loss / N

In [6]:
# function to compute the sigmoid loss
def softmax_loss(labeling=[], prediction=np.array([[]])):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    Returns the sigmoid loss score
    """
    N = len(labeling)
    loss = 0
    for i in range(N):
        cur_pred = prediction[i, labeling[i]]
        cur_num = np.exp(cur_pred)
        cur_den = np.sum(np.exp(prediction[i, :]))
        cur_softmax = cur_num / cur_den
        loss -= math.log(cur_softmax)
    return loss / N

In [7]:
# wrapper loss function
def loss(labeling=[], prediction=np.array([[]]), tau=0, loss_type='logloss'):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    tau: noise
    loss_type: logloss, sigmoidloss, or softmaxloss
    Returns the loss score
    """
    if loss_type == 'logloss':
        return k_ary_logloss(labeling, prediction) + tau * np.random.rand()
    if loss_type == 'sigmoidloss':
        return sigmoid_loss(labeling, prediction) + tau * np.random.rand()
    if loss_type == 'softmaxloss':
        return softmax_loss(labeling, prediction) + tau * np.random.rand()

In [8]:
# function to generate the predictions
def generate_prediction(N, K, tau_max):
    """
    N: number of data points
    K: number of classes
    tau_max: maximum noise
    Return a 2-d numpy array of predictions
    """
    prediction = np.zeros([N, K])
    for i in range(N):
        n = i + 1
        for j in range(K):
            k = j + 1
            prediction[i, j] = 2 ** ((n-1) * K + k) * N * tau_max
        
        diff = prediction[i, :] - max(prediction[i, :])
        prediction[i, :] = 3 ** diff
        prediction[i, :] /= sum(prediction[i, :])
        for j in range(K):
            prediction[i, j] = sys.float_info.min if prediction[i, j] < sys.float_info.min \
                else prediction[i, j]
    return prediction 

In [9]:
# generator for K-nary labelings
def gen_K_ary(nbits, K):
    """
    nbits: number of bits in the K-nary vector
    K: K in K-nary, e.g. K=2 is binary, K=3 is ternary
    Returns a generator for all K-nary labelings of length nbits
    """
    for i in range(K**nbits):
        bin_str = str(np.base_repr(i, K))
        l = len(bin_str)
        bin_arr = np.zeros(nbits)
        for j in range(l):
            bin_arr[nbits-j-1] = float(bin_str[l-j-1])
        yield [int(x) for x in bin_arr]

In [10]:
# the main label inference algorithm
def main_algorithm(noisy_loss, N, K, prediction, loss_type):
    """
    noisy_loss: observed loss score with noise
    N: number of data points
    K: number of classes
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    loss_type: either logloss or sigmoidloss
    Returns the inferred labeling
    """
    label_infer = []
    diff_min = np.inf
    lab_generator = gen_K_ary(N, K)
    
    while True:
        try:
            label_cur = next(lab_generator) 
            loss_cur = loss(label_cur, prediction, 0, loss_type)
            diff_cur = np.abs(loss_cur - noisy_loss) 
            if diff_cur < diff_min:
                label_infer = label_cur.copy()
                diff_min = diff_cur
        except:
            break
    return label_infer

In [11]:
# the main function for simulation
def main_sim(true_labeling, K, tau_max, loss_type, shard_size, max_N_feasible):
    """
    true_labeling: ground truth labeling
    K: number of classes
    tau_max: maximum noise
    loss_type: either logloss or sigmoidloss
    shard_size: number of data points in each shard
    max_N_feasible: largest N such that the label inference algorithm is feasible
    Returns the average accuracy and the number of shards
    """
    acc = 0
    mean_loss = 0
    n_total = len(true_labeling)  
    n_shard = int(np.ceil(n_total / shard_size))
    for j in range(n_shard):
        if j < n_shard - 1:
            cur_shard_size = shard_size
            labeling = true_labeling[j*shard_size : (j+1)*shard_size]
        else:
            cur_shard_size = n_total - j*shard_size
            labeling = true_labeling[j*shard_size : n_total]
        N = int(min(max_N_feasible, cur_shard_size))
        labeling_cur = labeling[:N]
        tau = 2 * np.exp(1) * tau_max / (np.exp(1) - 1)
        prediction = generate_prediction(N, K, tau)
        noisy_loss = loss(labeling_cur, prediction, tau_max, loss_type)
        labeling_infer = main_algorithm(noisy_loss, N, K, prediction, loss_type)
        acc += sum([x == y for x, y in zip(labeling_infer, labeling_cur)])
    return acc / n_total, n_shard

## label inference experiments

In [12]:
# logloss experiment
loss_type = 'logloss'
result_logloss = []

# read in the max_N_feasible for each K and tau_max setting from previous experimental results
setting_logloss = pd.read_csv('logloss_singlequery_mnist.csv')
 
df = setting_logloss.dropna().groupby(['K','tau_max'])['N'].max()
df = df.to_frame()
df = df.reset_index()
df.head()

K  tau_max  N
0  10   0.0001  3
1  10   0.0100  2
2  10   0.1000  2
3  10   1.0000  1

In [13]:
prev_acc = 0
for _, rows in df.iterrows():
    K, tau_max, max_N_feasible = int(rows[0]), rows[1], int(rows[2])
    shard_size_vec = [1,2,6,10,60,100,600,1000,6000,10000,30000,60000]
    shard_size_vec = list(reversed(shard_size_vec))
    
    n_total = len(cifar_label)
    for i, shard_size in enumerate(shard_size_vec):
        start = time.time()
        acc, n_shards = main_sim(cifar_label, K, tau_max, loss_type, shard_size, max_N_feasible)
        if i > 0:
            acc = max(acc, prev_acc)
        seconds = time.time() - start
        result_logloss.append((K, tau_max, acc, n_shards, seconds))
        print(K, tau_max, acc, n_shards, seconds)
        prev_acc = acc

10 0.0001 1.6666666666666667e-05 1 0.014931917190551758
10 0.0001 3.3333333333333335e-05 2 0.026394128799438477
10 0.0001 0.00011666666666666667 6 0.07059478759765625
10 0.0001 0.00018333333333333334 10 0.11301040649414062
10 0.0001 0.0013 60 0.6467878818511963
10 0.0001 0.002266666666666667 100 1.0130479335784912
10 0.0001 0.013716666666666667 600 6.0992982387542725
10 0.0001 0.0229 1000 10.270813941955566
10 0.0001 0.13946666666666666 6000 60.793092012405396
10 0.0001 0.23198333333333335 10000 101.00098371505737
10 0.0001 0.8486 30000 29.080676078796387
10 0.0001 1.0 60000 7.527997016906738
10 0.01 1.6666666666666667e-05 1 0.0011410713195800781
10 0.01 3.3333333333333335e-05 2 0.0020847320556640625
10 0.01 0.0001 6 0.00986170768737793
10 0.01 0.0002 10 0.016694068908691406
10 0.01 0.0012 60 0.08680486679077148
10 0.01 0.0019666666666666665 100 0.09881401062011719
10 0.01 0.0118 600 0.582697868347168
10 0.01 0.020033333333333334 1000 0.9738759994506836
10 0.01 0.11948333333333333 6000

In [14]:
df_logloss = pd.DataFrame(result_logloss)
df_logloss.columns = ['K', 'tau_max', 'acc', 'n_shards', 'seconds']
df_logloss.to_csv('logloss_multiquery_cifar.csv', index=False)
print(df_logloss.shape)
df_logloss.head(10)

(48, 5)


K  tau_max       acc  n_shards     seconds
0  10   0.0001  0.000017         1    0.014932
1  10   0.0001  0.000033         2    0.026394
2  10   0.0001  0.000117         6    0.070595
3  10   0.0001  0.000183        10    0.113010
4  10   0.0001  0.001300        60    0.646788
5  10   0.0001  0.002267       100    1.013048
6  10   0.0001  0.013717       600    6.099298
7  10   0.0001  0.022900      1000   10.270814
8  10   0.0001  0.139467      6000   60.793092
9  10   0.0001  0.231983     10000  101.000984

In [15]:
# softmaxloss experiment
loss_type = 'softmaxloss'
result_softmaxloss = []

# read in the max_N_feasible for each K and tau_max setting from previous experimental results
setting_softmaxloss = pd.read_csv('softmaxloss_singlequery_mnist.csv')
 
df = setting_softmaxloss.dropna().groupby(['K','tau_max'])['N'].max()
df = df.to_frame()
df = df.reset_index()

In [ ]:
for _, rows in df.iterrows():
    K, tau_max, max_N_feasible = int(rows[0]), rows[1], int(rows[2])
    shard_size_vec = [1,2,6,10,60,100,600,1000,6000,10000,30000,60000]
    shard_size_vec = list(reversed(shard_size_vec))
    
    n_total = len(cifar_label)
    for i, shard_size in enumerate(shard_size_vec):
        start = time.time()
        acc, n_shards = main_sim(cifar_label, K, tau_max, loss_type, shard_size, max_N_feasible)
        if i > 0:
            acc = max(acc, prev_acc)
        seconds = time.time() - start
        result_softmaxloss.append((K, tau_max, acc, n_shards, seconds))
        print(K, tau_max, acc, n_shards, seconds)
        prev_acc = acc

10 0.0001 1.6666666666666667e-05 1 0.0044171810150146484
10 0.0001 3.3333333333333335e-05 2 0.00978994369506836
10 0.0001 6.666666666666667e-05 6 0.02749800682067871
10 0.0001 0.0002 10 0.037631988525390625
10 0.0001 0.0010833333333333333 60 0.17833590507507324
10 0.0001 0.00185 100 0.30413007736206055
10 0.0001 0.010733333333333333 600 2.3211729526519775
10 0.0001 0.018133333333333335 1000 3.028917074203491
10 0.0001 0.10888333333333333 6000 18.614131927490234
10 0.0001 0.1815 10000 30.027349710464478
10 0.0001 0.5436 30000 89.93950605392456
10 0.0001 0.8969166666666667 60000 15.73885202407837
10 0.01 0.0 1 0.00047206878662109375
10 0.01 0.0 2 0.0007228851318359375
10 0.01 0.0 6 0.0020051002502441406
10 0.01 5e-05 10 0.002978086471557617
10 0.01 0.00013333333333333334 60 0.017009973526000977
10 0.01 0.0004 100 0.027102947235107422
10 0.01 0.0019166666666666666 600 0.15262103080749512
10 0.01 0.0035166666666666666 1000 0.28873491287231445
10 0.01 0.020383333333333333 6000 1.49374914169

In [ ]:
df_softmaxloss = pd.DataFrame(result_softmaxloss)
df_softmaxloss.columns = ['K', 'tau_max', 'acc', 'n_shards', 'seconds']
df_softmaxloss.to_csv('softmaxloss_multiquery_cifar.csv', index=False)
print(df_softmaxloss.shape)
df_softmaxloss.head(10)